<a href="https://colab.research.google.com/github/Yoonsun222/CapstoneDesign/blob/main/%EC%BA%A1%EC%8A%A4%ED%86%A4%EB%94%94%EC%9E%90%EC%9D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [78]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

df1  = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/CHILD_AIR_2022.01.31-02.06.csv", encoding = 'euc-kr', parse_dates=['등록일자'])
df2  = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/CHILD_AIR_2022.02.07-02.13.csv", encoding = 'euc-kr', parse_dates=['등록일자'])
df3  = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/CHILD_AIR_2022.02.14-02.20.csv", encoding = 'euc-kr', parse_dates=['등록일자'])
df4 = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/CHILD_AIR_2022.02.21-02.27.csv", encoding = 'euc-kr', parse_dates=['등록일자'])
df5 = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/CHILD_AIR_2022.02.28-03.06.csv", encoding = 'euc-kr', parse_dates=['등록일자'])
weather_df = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/기상청데이터.csv", encoding = 'euc-kr', parse_dates=['등록일자'])
dust_df = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/서울시미세먼지평균.csv",encoding = 'euc-kr', parse_dates=['등록일자'] )

df = pd.concat([df1,df2,df3,df4,df5],axis=0, join='inner')



In [79]:
df['등록일자'] = df["등록일자"].dt.strftime("%Y-%m-%d %H:00")

df

,시리얼,관측시각,온도(℃),습도(%),미세먼지(㎍/㎥),초미세먼지(㎍/㎥),휘발성유기화합물(ppb),이산화탄소(ppm),등록일자
0,V10G1802191,2.021050e+11,20,49,5,4,99,405,2022-01-31 00:00
1,V10G1802099,2.021050e+11,21,46,4,3,106,447,2022-01-31 00:00
2,V10G1802225,2.021050e+11,21,49,4,3,27,461,2022-01-31 00:00
3,V10G1802415,2.021050e+11,17,49,4,3,581,383,2022-01-31 00:00
4,V10G1802677,2.021050e+11,21,46,5,4,7,570,2022-01-31 00:00
...,...,...,...,...,...,...,...,...,...
642691,V10G1803626,2.021050e+11,22,49,3,2,143,703,2022-03-06 23:00
642692,V10G1802586,2.021050e+11,21,44,3,3,10,372,2022-03-06 23:00
642693,V10G1802051,2.021050e+11,21,49,4,3,25,69,2022-03-06 23:00
642694,V10G1802859,2.021050e+11,21,61,3,3,4,383,2022-03-06 23:00


In [80]:
weather_df['등록일자'] = weather_df["등록일자"].dt.strftime("%Y-%m-%d %H:00")

weather_df

,등록일자,기온(°C),풍속(m/s),습도(%),일조(hr),전운량(10분위)
0,2022-01-31 01:00,-4.6,2.4,51,0.0,0
1,2022-01-31 02:00,-5.0,2.3,52,0.0,0
2,2022-01-31 03:00,-5.3,1.6,55,0.0,0
3,2022-01-31 04:00,-5.8,1.0,55,0.0,0
4,2022-01-31 05:00,-6.1,1.9,58,0.0,0
...,...,...,...,...,...,...
811,2022-03-05 20:00,1.5,3.9,37,0.0,0
812,2022-03-05 21:00,0.7,3.3,40,0.0,0
813,2022-03-05 22:00,0.0,2.9,40,0.0,0
814,2022-03-05 23:00,-0.3,3.0,41,0.0,0


In [81]:
dust_df['등록일자'] = dust_df["등록일자"].dt.strftime("%Y-%m-%d %H:00")

dust_df

,등록일자,1시간평균 미세먼지농도(㎍/㎥)
0,2022-01-31 01:00,24
1,2022-01-31 02:00,26
2,2022-01-31 03:00,20
3,2022-01-31 04:00,28
4,2022-01-31 05:00,32
...,...,...
782,2022-03-05 18:00,36
783,2022-03-05 19:00,41
784,2022-03-05 21:00,33
785,2022-03-05 23:00,37


In [82]:
data = pd.merge(left = df , right = weather_df, how = "inner", on = "등록일자")
data = pd.merge(left = data , right = dust_df, how = "inner", on = "등록일자")

data.head(100)
data = data.drop_duplicates(['시리얼','등록일자'], keep='first')

data
#temp = df[df['시리얼']== 'V01T1622211']

,시리얼,관측시각,온도(℃),습도(%)_x,미세먼지(㎍/㎥),초미세먼지(㎍/㎥),휘발성유기화합물(ppb),이산화탄소(ppm),등록일자,기온(°C),풍속(m/s),습도(%)_y,일조(hr),전운량(10분위),1시간평균 미세먼지농도(㎍/㎥)
0,V10O1612855,2.022010e+11,0,41,65,24,-9999,-9999,2022-01-31 01:00,-4.6,2.4,51,0.0,0,24
2,V10O1612854,2.022010e+11,1,40,1,1,-9999,-9999,2022-01-31 01:00,-4.6,2.4,51,0.0,0,24
3,V10O1612850,2.022010e+11,0,42,38,14,-9999,-9999,2022-01-31 01:00,-4.6,2.4,51,0.0,0,24
4,V10O1612856,2.022010e+11,0,39,1,2,-9999,-9999,2022-01-31 01:00,-4.6,2.4,51,0.0,0,24
5,V10O1612853,2.022010e+11,0,41,30,11,-9999,-9999,2022-01-31 01:00,-4.6,2.4,51,0.0,0,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3397523,V10O1612861,2.022030e+11,3,31,27,9,-9999,-9999,2022-03-06 00:00,-0.6,3.4,43,0.0,0,36
3397524,V10O1612858,2.022030e+11,3,31,1,2,-9999,-9999,2022-03-06 00:00,-0.6,3.4,43,0.0,0,36
3397526,V01T1622211,2.022030e+11,21,15,23,17,129,318,2022-03-06 00:00,-0.6,3.4,43,0.0,0,36
3397527,V10O1612860,2.022030e+11,2,33,1,2,-9999,-9999,2022-03-06 00:00,-0.6,3.4,43,0.0,0,36


In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
data["시리얼"] = encoder.fit_transform(data["시리얼"])

data = data.reset_index()


In [84]:
total_data = data[data.columns[2:]]
total_data = total_data.replace(-9999,np.nan)
total_data = total_data.replace(-999,np.nan)
total_data = total_data.dropna(axis=0)
total_data.head(4)

total_data.columns = ['시간', '실내온도', '실내습도','실내미세먼지','실내초미세먼지','실내VOC','실내CO2','등록일자','외부기온','외부풍속','외부습도','일조','전운량','실외미세먼지농도']

In [85]:
corr_df= total_data.corr(method='pearson')

corr_df

total_data.to_csv('/content/drive/MyDrive/pre_capstone.csv',index=False)

In [86]:
total_data.sort_values(by='실내미세먼지')

X =  total_data[total_data.columns[[1,2,5,6,8,9,10,11,12,13]]]
Y = total_data[total_data.columns[[3]]]


In [93]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=222)
X_train, X_test, y_train, y_test = train_test_split(X_test, y_test, random_state=22)



scaler = MinMaxScaler()
X_scaled_train = scaler.fit_transform(X_train)
X_scaled_train = pd.DataFrame(X_scaled_train, columns=X_train.columns, index=X_train.index)

X_scaled_test = scaler.fit_transform(X_test)
X_scaled_test = pd.DataFrame(X_scaled_test, columns=X_test.columns, index=X_test.index)


X_scaled_test

y_test


,실내미세먼지
110955,4.0
178433,3.0
235291,6.0
77687,4.0
171338,0.0
...,...
181651,4.0
74257,5.0
241077,5.0
159322,11.0


In [ ]:
# X_scaled_train = np.reshape(X_scaled_train, (2055210, 10 , 2))

In [ ]:
from keras.layers import *
from keras.models import *
from keras.utils import *


model = Sequential()
model.add(LSTM(256, input_dim=1, input_length=10, stateful=True))
model.add(LSTM(64, return_sequences=True))
model.add(Dense(1, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()



In [ ]:
# hist = model.fit(X_train_scaled, Y_train, epochs=10, batch_size=1, validation_split=0.2)

In [ ]:
from sklearn.neural_network import MLPClassifier
model=MLPClassifier()
model.fit(X_scaled_train, y_train)
pred_train=model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)
pred_test=model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)

In [ ]:
joblib.dump(model, 'ml_class_model.pkl')

In [ ]:
from sklearn.neural_network import MLPRegressor
model=MLPRegressor(hidden_layer_sizes=(64,64,64),activation="relu" ,random_state=1, max_iter=100)
model.fit(X_scaled_train, y_train)
pred_train=model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)
pred_test=model.predict(X_scaled_test)



In [ ]:
import joblib

classifer = joblib.load("model.pkl")
new_observation = [[0.945312, 0.528090, 0.009385, 0.2620, 0.676, 0.256410, 0.521127, 1, 0, 0.489051]]

# 샘플의 클래스를 예측합니다.
classifer.predict(new_observation)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  "X does not have valid feature names, but"


array([9.46285273])

In [ ]:
model.score(X_scaled_test, y_test)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
model=KNeighborsRegressor()
model.fit(X_scaled_train, y_train)
pred_train=model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)
pred_test=model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)

In [ ]:


import numpy as np
from sklearn.metrics import mean_squared_error 676166
MSE_train = mean_squared_error(y_train, pred_train)
MSE_test = mean_squared_error(y_test, pred_test)
print("훈련   데이터 RMSE:", np.sqrt(MSE_train))
print("테스트 데이터 RMSE:", np.sqrt(MSE_test))


In [ ]:
param_grid={'n_neighbors': [1, 3, 5, 7, 9, 11]}
from sklearn.model_selection import GridSearchCV
grid_search=GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)
grid_search.fit(X_train_scaled, Y_train)
print("Best Parameter: {}".format(grid_search.best_params_))
print("Best Score: {:.4f}".format(grid_search.best_score_))
print("TestSet Score: {:.4f}".format(grid_search.score(X_test_scaled, Y_test)))